<a href="https://colab.research.google.com/github/dev-lusaja/03MAIR-Algoritmos-de-Optimizacion/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos(V2%2Cno_borrar).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Oscar Luis Sanchez Jara  <br>
Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---/tree/master/TrabajoPractico<br>
Google Colab: https://colab.research.google.com/drive/1TO32_2Pk5KL-Gp17tVuR7RL63rWqgVlx <br>
Problema:
>1. Sesiones de doblaje <br>
>2. Organizar los horarios de partidos de una jornada de La Liga<br>
>3. Configuración de Tribunales

Descripción del problema:


### Problema 1. Organizar sesiones de doblaje

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas.<br>

Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben.<br>

No es posible grabar más de 6 tomas por día.<br>

El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible.<br>
<br>
Los datos son:<br>
- Número de actores: 10<br>
- Número de tomas : 30<br>
<br>
Actores/Tomas : https://bit.ly/36D8IuK
- 1 indica que el actor participa en la toma
- 0 en caso contrario







                                        

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

#### **¿Como represento el espacio de soluciones?**
El espacio de soluciones se genera utilizando el metodo **create_poblation** el cual genera una poblacion de **100 individuos** donde cada uno es una lista de 30 elementos y su valor es el dia asignado para la toma.<br>

La cantidad de dias se calcula dividiendo la cantida de tomas entre las tomas maximas por dia.<br>

Y los individuos cumplen con la **restriccion** de las 6 tomas maximas por dia<br>

Ejemplo de individuo:
```
[1, 4, 0, 4, 0, 1, 4, 2, 4, 3, 3, 3, 0, 2, 3, 3, 1, 0, 4, 2, 2, 4, 3, 0, 2, 2, 0, 1, 1, 1]
```

#### **¿Cual es la función objetivo?**
En este problema queremos minimizar la cantidad total de días en los que trabajan los actores, cumpliendo con la restricción de que no se pueden grabar más de 6 tomas por día, para realizar esto nos apoyamos en 2 funciones **selection** y **fitness** que juntas permitiran formar esta formula<br>

$$
\min C = \sum_{i=1}^{N} D_i
$$

Donde:

- C es el costo total (cantidad de días distintos en los que trabajan los actores).
- Di es la cantidad de días diferentes en los que el actor "i" trabajó, usamos una máscara de bits para registrar los días y que sea mas sencillo de contabilizar.
- N es el número total de actores.


#### **¿Como implemento las restricciones?**

Las restricciones se implementaron como una validacion mediante la funcion **is_valid_individual**, que nos devolvera un valor booleano dependiendo si el individuo cumple con la restriccion.

$$
T_j \leq 6,  \forall j \in I
$$

donde:
- I es el conjunto de tomas
- Tj es cantidad de tomas asignadas al día en posicion "j"

In [13]:
#Respuesta
import pandas as pd
import numpy as np
import random

##############################
# Configuracion del problema #
##############################
'''
Utilizamos esta clase para almacenar configuracion que nos sera util en diferentes partes
'''
class Problem:
  @classmethod
  def set_assignment_matrix(cls, data):
    cls.assignment_matrix = data
    cls.num_shots         = cls.assignment_matrix.shape[0]
    cls.num_actors        = cls.assignment_matrix.shape[1]

    # Atributo que tiene el valor de las restricciones
    cls.max_shots_per_day = 6
    cls.total_days        = cls.num_shots // cls.max_shots_per_day

    # Atributos que se utilizaran en el algoritmo genetico
    cls.tournament_value  = 3
    cls.poblation_size    = 100
    cls.generations_size  = 100

    # Se utiliza este atributo para la probabilidad de cruce y mutacion
    cls.probability       = 0.5


#########################
# Función de evaluacion #
#########################
'''
Funcion que nos permitra obtener el costo de un individuo
'''
def fitness(individual):
  used_days = np.zeros(Problem.num_actors, dtype=int)
  for shot, day in enumerate(individual):
    for actor in range(Problem.num_actors):
      if Problem.assignment_matrix[shot][actor] == 1:
        # Marca el día en que trabaja el actor
        used_days[actor] |= (1 << day)

  # Costo por días de trabajo de los actores
  cost = sum(bin(d).count("1") for d in used_days)
  return cost

########################
# Función de seleccion #
########################
'''
Seleccionaremos el individuo con menor costo, utilizando la tecnica del torneo
'''
def selection(poblacion):
  return min(random.sample(poblacion, Problem.tournament_value), key=fitness)

###############
# Restriccion #
###############
'''
Implementamos una funcion que nos ayudara a verificar de acuerdo a la restriccion si el individuo es valido
'''
def is_valid_individual(individual):
  shots_per_day = np.zeros(Problem.total_days, dtype=int)
  for shot in individual:
    shots_per_day[shot] += 1
    if shots_per_day[shot] > Problem.max_shots_per_day:
      return False
  return True

#########################
# Espacio de soluciones #
#########################
'''
Metodo que nos permitira crear la poblacion inicial, tomando de la clase Problem
el tamaño de la poblacion
'''
def create_poblation():
  poblation = set()
  while len(poblation) < Problem.poblation_size:
    individual = create_individual()
    if is_valid_individual(individual):
      poblation.add(individual)
  return [list(ind) for ind in poblation]


#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

**Orden de complejidad**

Analizando el diseño propuesto podemos concluir que:
1. El número de tomas **S** y actores **A** son constantes dados por la hoja de calculo en comparación con el crecimiento de generaciones **G** y población **P**.
2. Las operaciones dentro de una **generación** (selección, cruce, mutación y evaluación) tienen una complejidad acotada por el tamaño de la población.

Bajo esta perspectiva, podríamos considerar la complejidad en términos de los dos factores dominantes:

$$
O(G.P)
$$

**Espacio de Soluciones**

El espacio de búsqueda está determinado por la cantidad de combinaciones posibles de asignaciones de tomas a días.<br>
Para cada una de las **S** tomas, hay **D** días posibles, por lo que el número total de combinaciones posibles es:

$$
D^{S}
$$

Este espacio puede crecer exponencialmente si el número de tomas aumenta, lo que hace que la búsqueda exhaustiva pueda ser un problema.<br>
Aquí es donde el algoritmo genético ayuda a explorar soluciones de manera **heurística** en lugar de recorrer todas las combinaciones posibles.

In [ ]:
#Respuesta


#Diseño
- **¿Que técnica utilizo? ¿Por qué?**

Se utilizo un **algoritmo genetico** para resolver el problema de asignación de tomas de doblaje, porque el problema tiene muchas combinaciones posibles de como asignar las tomas a dias, buscando minimizar el costo de los actores.

- **Detalles de la implementacion:**

1. La data que utilizamos la optenemos de una hoja de calculo que publico el profesor pero hemos agregado un control para que si la obtencion de esta data falla, la carguemos en un arreglo de numpy directamente.

2. Durante el diseño se construyeron funciones **geneticas** que ayudan a generar diferentes individuos y asi poder buscar el que tenga menor costo.

3. Para el caso de **Cruce** (cross) y **Mutacion** (mutation) se utilizo una probabilidad de que se pueda dar el cruce o mutacion con la finalidad de que podamos tener un porcentaje de individuos que pasen entre generaciones. Estas funciones utilizan el metodo **is_valid_individual** para garantizar que estan generando un individuo correcto.

4. Nuestra funcion **genetic_algorithm** es la encargada de orquestar todas las funiones creadas y empezar a iterar entre generaciones y poblacion buscando el individuo mas optimo.

5. Finalmente mostramos el resultado en una impresion en pantalla y tambien mediante un dataframe de pandas para hcerlo mas visual.

In [22]:
#Respuesta

##########################
# Cargar datos desde CSV #
##########################
'''
Intentamos cargar la hoja de calculo dada por el profesor y si no es posible
tomamos los valores que hemos almacenado en un array de numpy queson exactamente los mismos
'''
def load_data():
  try:
    url = "https://docs.google.com/spreadsheets/d/1Ipn6IrbQP4ax8zOnivdBIw2lN0JISkJG4fXndYd27U0/export?format=csv"
    df = pd.read_csv(url)
    assignment_matrix = df.iloc[1:31, 1:11].to_numpy()
  except:
    assignment_matrix = np.array([
        [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 1., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 1., 0., 1., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 1., 1., 0., 0.],
        [0., 1., 0., 1., 0., 0., 0., 1., 0., 0.],
        [1., 1., 0., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 1., 0., 0., 0., 0.],
        [1., 1., 0., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 1., 0., 0., 1., 0.],
        [1., 1., 1., 0., 1., 0., 0., 1., 0., 0.],
        [1., 1., 1., 1., 0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 1., 1., 0., 0., 0., 0., 0.],
        [1., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 1.],
        [1., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
        [1., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 1., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 1., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
        [1., 1., 0., 1., 0., 0., 0., 0., 0., 1.],
        [1., 0., 1., 0., 1., 0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 1., 1., 0., 0., 0., 0.],
        [1., 0., 0., 1., 0., 0., 0., 0., 0., 0.]
    ])
  finally:
    return assignment_matrix


#######################
# Funciones geneticas #
#######################

def create_individual():
    individual = tuple(random.choices(range(Problem.total_days), k=Problem.num_shots))
    return individual

def cross(father1, father2):
  son1 = father1
  son2 = father2
  if random.random() < Problem.probability:
    point = random.randint(1, Problem.num_shots - 2)
    individual = father1[:point] + father2[point:]
    if is_valid_individual(individual):
      son1 = individual
  if random.random() < Problem.probability:
    point = random.randint(1, Problem.num_shots - 2)
    individual2 = father2[:point] + father1[point:]
    if is_valid_individual(individual2):
      son2 = individual2
  return son1, son2

def mutation(individual):
  individual = list(individual)
  original = individual[:]
  for i in range(len(individual)):
    if random.random() < Problem.probability:
      individual[i] = random.randint(0, Problem.total_days - 1)
    if is_valid_individual(individual):
      return tuple(individual)
  return tuple(original)


######################
# Algoritmo genetico #
######################

def genetic_algorithm():
  # Inicializamos la matriz de asignacion con la data obtenida
  Problem.set_assignment_matrix(load_data())

  # Creamos posibles soluciones
  poblation = create_poblation()

  for _ in range(Problem.generations_size):
      new_poblation = []
      for _ in range(Problem.poblation_size // 2):
        father1 = selection(poblation)
        father2 = selection(poblation)
        son1, son2 = cross(father1, father2)
        new_poblation.extend([mutation(son1), mutation(son2)])
      poblation = sorted(new_poblation, key=fitness)[:Problem.poblation_size]

  return poblation[0]

best_solution = genetic_algorithm()
print(f"\nLa mejor solución encontrada es:\n{best_solution}")

print(f"\nVisualizacion de resultados usando un dataframe\n")
df = pd.DataFrame({'Tomas': range(1, len(best_solution) + 1), 'Dias': [d + 1 for d in best_solution]})
df_grouped = df.groupby('Dias')['Tomas'].apply(list).to_frame()
display(df_grouped)



Mejor solución encontrada es:
(4, 4, 3, 3, 0, 0, 4, 1, 0, 3, 4, 1, 2, 3, 3, 2, 1, 1, 3, 2, 0, 1, 2, 2, 1, 4, 2, 4, 0, 0)

Visualizacion de resultados usand un dataframe



,Tomas
Dias,
1,"[5, 6, 9, 21, 29, 30]"
2,"[8, 12, 17, 18, 22, 25]"
3,"[13, 16, 20, 23, 24, 27]"
4,"[3, 4, 10, 14, 15, 19]"
5,"[1, 2, 7, 11, 26, 28]"
